### Mapping Hate Crime Data

In this notebook, we'll be working toward using this large dataset to present a LOT of data per-precinct using a datawrapper map of NYPD precincts paired to our dataset's `Complaint Precinct Code` column. You'll need a datawrapper account to generate your own.

In [1]:
import pandas as pd 
import math 
import re 
import datawrapper 
from IPython.display import HTML

Grab an API key for your datawrapper personal (free!) account before continuing. Instructions are available on [their website here](https://developer.datawrapper.de/docs/getting-started).

In [2]:
API_TOKEN = "" #replace with your API Key 

In [3]:
#helper function because datawrapper hates anything but alphanumeric snakecase for the tooltip variables 
#https://en.wikipedia.org/wiki/Snake_case 
def to_snake_case(text):
    # Converts column names to datawrapper-compatible format
    # e.g. "Ethnicity/National Origin/Ancestry" -> "ethnicity_national_origin_ancestry"
    # and "60 YRS AND OLDER" -> "n_60_yrs_and_older"
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    result = re.sub(r'[\s_]+', '_', text.lower()).strip('_')
    return f"n_{result}" if result[0].isdigit() else result

Since we're in a new notebook, we need to load up our prepared dataset.

In [4]:
hate_crimes_over_time = pd.read_csv('./output/hate_crimes_w_codecats.csv')

#### calculator functions 
These are the same calculator functions we've been using in our main notebook. 

In [5]:
def calc_stats(df):
    total = len(df)
    violent_incidents = len(df[df['Violent'] == 1])
    violent_arrests = len(df[(df['Violent'] == 1) & ((df['Arrest Date'].notna()) | (df['Arrest Id'].notna()))])
    non_violent_arrests = len(df[(df['Violent'] == 0) & ((df['Arrest Date'].notna()) | (df['Arrest Id'].notna()))])
    total_arrests = violent_arrests + non_violent_arrests
    
    return {
        'Total': total,
        'Violent': violent_incidents,
        '% Violent': round(violent_incidents / total * 100, 2) if total > 0 else 0,
        'Non-Violent Arrests': non_violent_arrests,
        'Violent Arrests': violent_arrests,
        'Total Arrests': total_arrests,
        '% Arrests': round(total_arrests / total * 100, 2) if total > 0 else 0,
        '% Violent Arrests': round(violent_arrests / violent_incidents * 100, 2) if violent_incidents > 0 else 0
    }
    
def calc_by_cat(df, category_column='Offense Category', group_column='Complaint Precinct Code'):
    categories = sorted(df[category_column].unique())
    all_metrics = []
    
    for precinct in df[group_column].unique():
        precinct_data = df[df[group_column] == precinct]
        precinct_metrics = {'Precinct code': precinct}
        overall_metrics = calc_stats(precinct_data)
        precinct_metrics.update({
            'total_incidents': overall_metrics['Total'],
            'violent_incidents': overall_metrics['Violent'],
            'violent_percentage': overall_metrics['% Violent'],
            'arrests': overall_metrics['Total Arrests'],
            'arrest_percentage': overall_metrics['% Arrests']
        })
        
        for category in categories:
            cat_snake = to_snake_case(category)
            cat_df = precinct_data[precinct_data[category_column] == category]
            cat_metrics = calc_stats(cat_df)
            precinct_metrics.update({
                f'{cat_snake}_incidents': cat_metrics['Total'],
                f'{cat_snake}_percentage': round(cat_metrics['Total'] / overall_metrics['Total'] * 100, 1) if overall_metrics['Total'] > 0 else 0,
                f'{cat_snake}_violent_percentage': cat_metrics['% Violent'],
                f'{cat_snake}_arrest_percentage': cat_metrics['% Arrests']
            })
        
        all_metrics.append(precinct_metrics)
    
    metrics_df = pd.DataFrame(all_metrics)
    return metrics_df, {cat: to_snake_case(cat) for cat in categories}

#### mapping to datawrapper

This function preps metrics that we'll use in our choropleth visualization of the hate crime data by running our calculator functions against specific categories and for every precinct in the dataset. 

- We pull a column from our imported dataframe (`category_column`) to use for the tooltip breakdown of additional statistics per-precinct.
- We calculate the total incidents per precinct by counting the unique rows in the `Full Complaint ID` column.
- Similarly, we then use the value in `Violent` to caluclate both a violent incident count and percentage per precinct.
- Using the same method, we calculate the number of non-empty `Arrest Id` values per row per precinct for our arrest count and arrest percentage.

Then, using a **for** loop, we take every variable from the `category_column` variable column and run each through the same calculations - incidents, violence, arrests and percentages, all sorted by precinct code and then by category. 

In [6]:
def calculate_hate_crimes_by_category(df, category_column='Offense Category', group_column='Complaint Precinct Code'):
    categories = sorted(df[category_column].unique())
    category_mapping = {cat: to_snake_case(cat) for cat in categories}
    
    all_precincts = sorted(df[group_column].unique())
    metrics_list = []
    
    for precinct in all_precincts:
        precinct_df = df[df[group_column] == precinct]
        precinct_stats = calc_stats(precinct_df)
        row_data = {
            'Precinct code': precinct,
            'total_incidents': precinct_stats['Total'],
            'violent_incidents': precinct_stats['Violent'],
            'violent_percentage': precinct_stats['% Violent'],
            'arrests': precinct_stats['Total Arrests'],
            'arrest_percentage': precinct_stats['% Arrests']
        }
        
        for category in categories:
            cat_snake = to_snake_case(category)
            cat_df = precinct_df[precinct_df[category_column] == category]
            cat_stats = calc_stats(cat_df)
            
            row_data.update({
                f'{cat_snake}_incidents': cat_stats['Total'],
                f'{cat_snake}_percentage': round(cat_stats['Total'] / precinct_stats['Total'] * 100, 1) if precinct_stats['Total'] > 0 else 0,
                f'{cat_snake}_violent_percentage': cat_stats['% Violent'],
                f'{cat_snake}_violent_count': cat_stats['Violent'],  
                f'{cat_snake}_arrest_percentage': cat_stats['% Arrests'],
                f'{cat_snake}_arrest_count': cat_stats['Total Arrests'] 
            })
        
        metrics_list.append(row_data)
    
    metrics = pd.DataFrame(metrics_list)
    
    return metrics, category_mapping

This function creates our choropleth visualization using datawrapper's library through their API. The focus of most of this function is on the advanced tooltip we're prepping based on Datawrapper's guide to [Embedded Graphs in Tooltips](https://academy.datawrapper.de/article/282-how-to-embed-charts-into-tooltips). 

<img src="https://d33v4339jhl8k0.cloudfront.net/docs/assets/588089eddd8c8e484b24e90a/images/6155dae012c07c18afdd93b8/file-BCaBsmKANx.png" width="75%">

- We loop through each category pulled from our selected `category_column` to prep html and datawrapper syntax for the tooltip for matching barcharts for the incident count, violent incidents and arrests.
- We prep some basic `chart_config` settings for datawrapper that can be edited on their website later.
- We set the data source (though this can be edited later)
- Crucially, we tell datawrapper which basemap we're going to be using and how to pair that basemap to our data (in this case, by precinct code).

**Note:** Datawrapper only allows a limited [subset of HTML tags](https://academy.datawrapper.de/article/237-i-want-to-change-how-my-data-appears-in-tooltips) be used in the tooltip section and only for certain kinds of charts, including maps and scatterplots: 

```html
<a> <abbr> <address> <audio> <b> <big> <blockquote> <br/> <br> <caption> <cite> <code> <col> <colgroup> <dd> <del> <details> <dfn> <div> <dl> <dt> <em> <figure> <h1> <h2> <h3> <h4> <h5> <h6> <hr> <hgroup> <i> <img> <ins> <kbd> <li> <mark> <meter> <ol> <p> <pre> <q> <s> <small> <span> <strike> <strong> <sub> <summary> <sup> <table> <tbody> <td> <th> <thead> <tfoot> <tr> <tt> <u> <ul> <wbr><br>
```

So you can't start building a form or embedding additional multimedia inside of your tooltip, but you *can* still get creative. 

Here's what one of Datwrapper's examples looks like in html: 

```html
<div> <div style="margin-bottom:11px;"> <div style='margin-bottom:4px;'>Women</div> <div style='background:#F2F2F2; width: 150px; height:15px; display:flex; align-items: center;'> <div style='display:flex; align-items:center; background:#BFA817; height:100%; width: {{women}}%'> {{ women >= 20 ? CONCAT("<span style='color:white; font-weight:bold;margin:0px 5px;'>", FORMAT(women, '0'), "%</span>") : "" }} </div> 	{{ women < 20 ? CONCAT("<span style='color:black; font-weight:bold;margin:0px 5px;'>", FORMAT(women, '0'), "%</span>") : "" }} </div> </div> <div> <div style='margin-bottom:4px;'>Men</div> <div style='background:#F2F2F2; width: 150px; height:15px; display:flex; align-items: center;'> <div style='display:flex; align-items:center; background:#7AAFCB; height:100%; width: {{ men }}%'> {{ men >= 20 ? CONCAT("<span style='color:white; font-weight:bold;margin:0px 5px;'>", FORMAT(men, '0'), "%</span>") : "" }} </div> 	{{ men < 20 ? CONCAT("<span style='color:black; font-weight:bold;margin:0px 5px;'>", FORMAT(men, '0'), "%</span>") : "" }} </div> </div> </div><br>
```

For our purposes, we're generating html for the tooltip section that will end up looking something like this (using the `age` category as an example):   

```html
{{ age_incidents > 0 ? CONCAT("<div style=margin-bottom:16px><div style=margin-bottom:4px><b>Age</b></div><div style=display:flex><div style=background:#4e79a7;width:", FORMAT(age_percentage * 2, "0"), "px>", age_percentage >= 80 ? CONCAT("<span style=color:white;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>", FORMAT(age_percentage, "0"), "%</span>") : "" ,"</div><div style=background:#F2F2F2;width:", FORMAT(200-(age_percentage * 2), "0"), "px>", age_percentage < 80 ? CONCAT("<span style=color:#4e79a7;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>", FORMAT(age_percentage, "0"), "%</span>") : "" ,"</div></div><div style=font-size:12px;color:#666;margin-top:2px>", age_incidents, " incidents</div><div style=display:flex;margin-top:8px><div style=background:#ff4444;width:", FORMAT(age_violent_percentage * 2, "0"), "px>", age_violent_percentage >= 80 ? CONCAT("<span style=color:white;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>", FORMAT(age_violent_percentage, "0"), "%</span>") : "" ,"</div><div style=background:#F2F2F2;width:", FORMAT(200-(age_violent_percentage * 2), "0"), "px>", age_violent_percentage < 80 ? CONCAT("<span style=color:#ff4444;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>", FORMAT(age_violent_percentage, "0"), "%</span>") : "" ,"</div></div><div style=font-size:12px;color:#666;margin-top:2px>", age_violent_count, " violent incidents</div><div style=display:flex;margin-top:8px><div style=background:#2ca02c;width:", FORMAT(age_arrest_percentage * 2, "0"), "px>", age_arrest_percentage >= 80 ? CONCAT("<span style=color:white;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>", FORMAT(age_arrest_percentage, "0"), "%</span>") : "" ,"</div><div style=background:#F2F2F2;width:", FORMAT(200-(age_arrest_percentage * 2), "0"), "px>", age_arrest_percentage < 80 ? CONCAT("<span style=color:#2ca02c;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>", FORMAT(age_arrest_percentage, "0"), "%</span>") : "" ,"</div></div><div style=font-size:12px;color:#666;margin-top:2px>", age_arrest_count, " resulted in arrest</div></div>") : "" }}

```

In our function, we loop through every category variable to generate a version of this code block which: 
- scales our bar graphs to fill as a percentage of the total bar (200px), because we're kind of hacking a bar graph together we're generating the color and the grey section of the bar separately with `FORMAT(age_percentage * 2, "0")` and `FORMAT(200-(age_percentage * 2), "0")`
- pulling variable data we calculated for each category and calling it both as text and as the dimensions of each bar graph. 

Within the interactive, that will display as a dynamic tooltip like this below. 

<img src="../reference/tooltip.png" width="220px">

In addition to bog-standard html, in our function below you'll see that we're using an if-else conditional here for each category (`{{ age_incidents > 0 ? CONCAT(` etc as above). Datawrapper uses js-y ternary conditional syntax for their tooltips, which allows us to do some helpful things. 

Datawrapper explains a little bit about their if else statement syntax for tooltips in [this article](https://academy.datawrapper.de/article/237-i-want-to-change-how-my-data-appears-in-tooltips). If you want to experiment with it, you might also reference GeeksforGeeks' [JS Ternary Operator tutorial too](https://www.geeksforgeeks.org/javascript-ternary-operator/).

In this case, we're telling the tooltip settings to check each variable we prepare to see if the total incident number for that category is greater than zero. 

```js 
category > 0 ? CONCAT
```

If it is, it will shove our custom html together and display the sub-bar charts for that category. If it isn't, it won't show that category for that precinct at all. This is especially helpful for the rarest categories in the dataset which would otherwise display blank 0% bars over and over. 


**NOTE**: For this map, we're using datawrapper's built in basemap for NYC precinct data, or `us-nyc-policeprecincts`. If you're not sure the shorthand for the basemap you need you can run [this request](https://developer.datawrapper.de/reference/getbasemaps) in a terminal to pull the full list: 

```shell
curl --request GET \
     --url https://api.datawrapper.de/v3/basemaps \
     --header 'accept: */*'
```

Your dataset must have a key column to associated with the basemap's geographical data. In this case, our dataset has a column of precinct codes that matches the precinct basemap. 

In [7]:
def create_offense_category_map(df, category_mapping, api_token, title, description):
    dw = datawrapper.Datawrapper(access_token=api_token)
    
    fields = {
        "precinct_code": "Precinct code",
        "total_incidents": "total_incidents",
        "violent_incidents": "violent_incidents",
        "violent_percentage": "violent_percentage",
        "arrests": "arrests",
        "arrest_percentage": "arrest_percentage"
    }
    
    for cat_snake in category_mapping.values():
        for suffix in ["incidents", "percentage", "violent_percentage", "arrest_percentage"]:
            fields[f"{cat_snake}_{suffix}"] = f"{cat_snake}_{suffix}" #adds our calculated fields as fields datawrapper can call on in the tooltip 
    
    category_sections = []
    for orig_cat, cat_snake in category_mapping.items():
        category_sections.append( #this becomes the html in the tooltip section of datawrapper's visualization editor - we'll generate this same codeblock for every category in our category column and then append them all after our precinct data intro.
            f'{{{{ {cat_snake}_incidents > 0 ? CONCAT('\
                f'\"<div style=margin-bottom:16px>'\
                    f'<div style=margin-bottom:4px>'\
                        f'<b>{orig_cat}</b>'\
                    f'</div>'\
                    
                    # Incidents bargraph
                    f'<div style=display:flex>'\
                        f'<div style=background:#4e79a7;width:\", FORMAT({cat_snake}_percentage * 2, \"0\"), \"px>'\
                            f'\", {cat_snake}_percentage >= 80 ? CONCAT(\"<span style=color:white;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>'\
                                f'\", FORMAT({cat_snake}_percentage, \"0\"), \"%</span>\") : \"\" ,\"'\
                        f'</div>'\
                        f'<div style=background:#F2F2F2;width:\", FORMAT(200-({cat_snake}_percentage * 2), \"0\"), \"px>'\
                            f'\", {cat_snake}_percentage < 80 ? CONCAT(\"<span style=color:#4e79a7;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>'\
                                f'\", FORMAT({cat_snake}_percentage, \"0\"), \"%</span>\") : \"\" ,\"'\
                        f'</div>'\
                    f'</div>'\
                    f'<div style=font-size:12px;color:#666;margin-top:2px>'\
                        f'\", {cat_snake}_incidents, \" incidents'\
                    f'</div>'\
                    
                    # Violent incidents bargraph
                    f'<div style=display:flex;margin-top:8px>'\
                        f'<div style=background:#ff4444;width:\", FORMAT({cat_snake}_violent_percentage * 2, \"0\"), \"px>'\
                            f'\", {cat_snake}_violent_percentage >= 80 ? CONCAT(\"<span style=color:white;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>'\
                                f'\", FORMAT({cat_snake}_violent_percentage, \"0\"), \"%</span>\") : \"\" ,\"'\
                        f'</div>'\
                        f'<div style=background:#F2F2F2;width:\", FORMAT(200-({cat_snake}_violent_percentage * 2), \"0\"), \"px>'\
                            f'\", {cat_snake}_violent_percentage < 80 ? CONCAT(\"<span style=color:#ff4444;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>'\
                                f'\", FORMAT({cat_snake}_violent_percentage, \"0\"), \"%</span>\") : \"\" ,\"'\
                        f'</div>'\
                    f'</div>'\
                    f'<div style=font-size:12px;color:#666;margin-top:2px>\", {cat_snake}_violent_count, \" violent incidents</div>'\
                    
                    # Arrests bargraph
                    f'<div style=display:flex;margin-top:8px>'\
                        f'<div style=background:#2ca02c;width:\", FORMAT({cat_snake}_arrest_percentage * 2, \"0\"), \"px>'\
                            f'\", {cat_snake}_arrest_percentage >= 80 ? CONCAT(\"<span style=color:white;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>'\
                                f'\", FORMAT({cat_snake}_arrest_percentage, \"0\"), \"%</span>\") : \"\" ,\"'\
                        f'</div>'\
                        f'<div style=background:#F2F2F2;width:\", FORMAT(200-({cat_snake}_arrest_percentage * 2), \"0\"), \"px>'\
                            f'\", {cat_snake}_arrest_percentage < 80 ? CONCAT(\"<span style=color:#2ca02c;font-weight:bold;padding:4px 4px 0px 4px;display:inline-block>'\
                                f'\", FORMAT({cat_snake}_arrest_percentage, \"0\"), \"%</span>\") : \"\" ,\"'\
                        f'</div>'\
                    f'</div>'\
                    f'<div style=font-size:12px;color:#666;margin-top:2px>\", {cat_snake}_arrest_count, \" resulted in arrest</div>'\
                f'</div>\") : \"\" }}}}'\
            )

    tooltip_template = { #this does the initial setup for the tooltip with our intro which will summarize the hate crime data for each precinct
        "title": "<big>Precinct {{ precinct_code }}</big>",
        "body": (
                    '<b>{{ total_incidents }}</b> hate crimes were reported. <b>{{ violent_incidents }}</b> were for allegedly violent crimes. <b>{{ arrests }}</b> resulted in arrest.'\
                '<hr style="border-top: 1px solid #CFCFCF;width:400px;margin-left:-20px">'\
                '<div style=margin-bottom:12px>'\
                    '<b><big>Breakdown by Category</big></b>'\
                '</div>'\
                + ''.join(category_sections) +\
            '</div>'\
        ),
        "fields": fields
    }

    chart_config = { #everything here can be edited after you create the graph back in datawrapper's UI, we're just skipping making the data work and writing the tooltip code manually
        "describe": {
            "intro": description,
            "source-name": "NYPD Data",
            "source-url": "https://data.cityofnewyork.us"
        },
        "visualize": {
            "basemap": "us-nyc-policeprecincts",
            "map-key-column": "Precinct code",
            "map-value-column": "total_incidents",
            "map-color-scale": "Blues",
            "tooltip": tooltip_template
        },
        "data": {
            "column-format": {
                "Precinct code": {"type": "text", "role": "key"}, 
                "total_incidents": {"type": "number", "role": "value"}
            }
        }
    }
    
    chart = dw.create_chart(title=title, chart_type='d3-maps-choropleth') #you can call this variable from datawrapper to see all your options for other graphs
    chart_id = chart['id']
    
    dw.update_chart(
        chart_id=chart_id,
        title=title,
        metadata=chart_config
    )
    
    dw.add_data(chart_id, df.to_csv(index=False))
    
    return chart_id, df

Now we apply our dynamic calculator to our precinct data and generate our map. 

In [8]:
precinct_metrics, category_mapping = calculate_hate_crimes_by_category(
    hate_crimes_over_time, 
    category_column='Offense Category'
)

In [9]:
chart_id, df = create_offense_category_map(
    df=precinct_metrics,
    category_mapping=category_mapping,
    api_token=API_TOKEN,
    title="DRAFT: NYPD Hate Crime Data by Precinct and Target Category - Python Gen",
    description="Every hate crime in New York City reported by the New York City Police Department from Jan 2019- Sep 2024. <br> Scroll to see <b style='color:white; background-color:#2468f0;box-decoration-break:clone;-webkit-box-decoration-break: clone;padding:0px 3px;'>total</b> reported incidents, <b style='color:white; background-color:#ff1744;box-decoration-break:clone;-webkit-box-decoration-break: clone;padding:0px 3px;'>violent</b> hate crimes and <b style='color:white; background-color:#2ca02c;box-decoration-break:clone;-webkit-box-decoration-break: clone;padding:0px 3px;'>arrests</b> broken down by bias category.<br><br>"
)

print(f"Created chart with ID: {chart_id}")
print(f"https://app.datawrapper.de/edit/{chart_id}/visualize#refine")

Created chart with ID: tryE9
https://app.datawrapper.de/edit/tryE9/visualize#refine


You can push the chart to publish from the datawrapper python library but for this workflow, copy and paste the link to your logged-in browser and doublecheck your final settings in the web interface. Once you confirm "publish" in the web interface, run the next cell with your `chart_id` swapped in to see our map. 

In [17]:
HTML('<iframe title="DRAFT: NYPD Hate Crime Data by Precinct and Target Category - Python Gen" aria-label="Map" id="datawrapper-chart-tryE9" src="https://datawrapper.dwcdn.net/tryE9/1/" scrolling="no" frameborder="0" style="width: 0; min-width: 100% !important; border: none;" height="825" data-external="1"></iframe><script type="text/javascript">!function(){"use strict";window.addEventListener("message",(function(a){if(void 0!==a.data["datawrapper-height"]){var e=document.querySelectorAll("iframe");for(var t in a.data["datawrapper-height"])for(var r=0;r<e.length;r++)if(e[r].contentWindow===a.source){var i=a.data["datawrapper-height"][t]+"px";e[r].style.height=i}}}))}();</script>')

Github blocks HTML rendering so if you're previewing this notebook on Github, you can see an example of what we've generated here: 

<img src="../reference/choropleth_demo.gif" width="700px"><br> 